In [1]:
from A.preProcessing import *
from A.word2vecProcessing import trainingEmbedding, createCluster
from A.exam_CRF import evaluate
from A.contextProcessing import createAllForPol
from A.polClassification_ML import examByML, getFeaturesAndPolsFromFile


import os, sys
JAVA_HOME = r"D:\Program Files\Java\jdk-22\bin\java.exe"
os.environ.setdefault('JAVA_HOME', JAVA_HOME)

'D:\\Program Files\\Java\\jdk-22\\bin\\java.exe'

In [2]:
filepath='B/contextFiles/re_train.cox'
d_type = 're'
trainX,trainY,testX,testY=getFeaturesAndPolsFromFile(filepath,d_type,0.8)

2024-04-25 18:12:02,883:INFO:loading Word2Vec object from B/model/Restaurants.w2v


2024-04-25 18:12:02,891:INFO:loading wv recursively from B/model/Restaurants.w2v.wv.* with mmap=None
2024-04-25 18:12:02,891:INFO:loading vectors from B/model/Restaurants.w2v.wv.vectors.npy with mmap=None
2024-04-25 18:12:02,912:INFO:loading syn1neg from B/model/Restaurants.w2v.syn1neg.npy with mmap=None
2024-04-25 18:12:02,929:INFO:setting ignored attribute cum_table to None


Loading context information from B/contextFiles/re_train.cox
Loading dataset...


2024-04-25 18:12:03,102:INFO:Word2Vec lifecycle event {'fname': 'B/model/Restaurants.w2v', 'datetime': '2024-04-25T18:12:03.102359', 'gensim': '4.3.2', 'python': '3.9.19 (main, Mar 21 2024, 17:21:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'loaded'}


Finish reading word2vec


In [21]:
trainX

array([[-0.03359752,  0.03154179,  0.02124864, ...,  0.01777572,
         0.00078053, -0.02552287],
       [-0.05147164,  0.02440695,  0.02206931, ...,  0.0031999 ,
         0.02163599,  0.00562235],
       [-0.05497746,  0.00409167,  0.02364825, ...,  0.01984054,
         0.05254352,  0.00450022],
       ...,
       [-0.07138217,  0.02533806,  0.02860931, ..., -0.01924449,
         0.01923998,  0.03583211],
       [-0.01187075,  0.01720975,  0.04604203, ...,  0.00218696,
         0.06882735,  0.00759861],
       [-0.08047906,  0.00263159,  0.0355151 , ..., -0.04804308,
         0.05078454,  0.00541531]])

In [4]:
import torch
t = torch.Tensor(trainX[0])

In [23]:
X = trainX.reshape(-1, 1, 20, 30)
torch.tensor(X, dtype=torch.float32)

tensor([[[[-3.3598e-02,  3.1542e-02,  2.1249e-02,  ..., -5.6684e-02,
           -7.5319e-04, -4.2134e-02],
          [-2.5824e-02,  1.5820e-02,  5.0212e-03,  ..., -3.4679e-02,
           -6.5350e-02,  4.4906e-02],
          [-1.2635e-02,  8.7255e-02,  2.4547e-02,  ..., -2.2766e-02,
            2.0042e-02, -2.5810e-02],
          ...,
          [ 4.0504e-02, -2.5263e-02, -1.8841e-02,  ..., -1.5819e-02,
            3.3649e-02,  1.3912e-02],
          [ 1.9313e-02, -1.7787e-03, -1.8701e-02,  ..., -5.6629e-02,
            3.6553e-02, -2.4077e-03],
          [-6.7818e-03, -3.4592e-02,  4.6433e-02,  ...,  1.7776e-02,
            7.8053e-04, -2.5523e-02]]],


        [[[-5.1472e-02,  2.4407e-02,  2.2069e-02,  ..., -3.7273e-02,
           -3.2071e-02, -4.2251e-02],
          [-3.2134e-02,  4.8145e-02, -2.8855e-02,  ..., -3.1376e-02,
           -4.8383e-02,  2.0164e-02],
          [ 2.2582e-02,  4.3350e-02,  3.0072e-02,  ..., -2.1196e-02,
            1.5166e-02, -3.4534e-02],
          ...,
   

In [29]:
X = trainX.reshape(-1)
torch.tensor(X, dtype=torch.float32)

tensor([-0.0336,  0.0315,  0.0212,  ..., -0.0480,  0.0508,  0.0054])

In [5]:
t.view(20, 30)

tensor([[-0.0336,  0.0315,  0.0212,  0.0372,  0.0130, -0.0428,  0.0095,  0.0174,
          0.0332, -0.0050, -0.0079,  0.0267,  0.0172,  0.0032,  0.0024, -0.0316,
          0.0251,  0.0145, -0.0146, -0.0189,  0.0015,  0.0173, -0.0529,  0.0461,
          0.0391, -0.0456,  0.0303, -0.0567, -0.0008, -0.0421],
        [-0.0258,  0.0158,  0.0050,  0.0096, -0.0004,  0.0134, -0.0457,  0.0691,
          0.0248,  0.0392, -0.0211, -0.0256,  0.0101,  0.0303, -0.0812, -0.0094,
         -0.0132, -0.0136, -0.0186, -0.0004,  0.0185, -0.0007, -0.0153,  0.0332,
         -0.0433, -0.0315,  0.0360, -0.0347, -0.0654,  0.0449],
        [-0.0126,  0.0873,  0.0245, -0.0358,  0.0413, -0.0094, -0.0085, -0.0136,
         -0.0025,  0.0344,  0.0415, -0.0546, -0.0196,  0.0311,  0.0394, -0.0736,
          0.0262, -0.0270, -0.0262, -0.0153,  0.0379,  0.0501,  0.0299, -0.0473,
         -0.0063,  0.0429, -0.0421, -0.0228,  0.0200, -0.0258],
        [ 0.0070, -0.0420, -0.0175, -0.0124, -0.0525,  0.0534, -0.0335, -0.0494

In [5]:
from A.nnModels import create_dataloader

In [6]:
train_dataset = create_dataloader(trainX, trainY)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [15]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output size: (16, 10, 15)
        
        # Second convolutional layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output size: (32, 5, 7)
        
        # Fully connected layer
        self.fc1 = nn.Linear(32 * 5 * 7, 120)  # Input features are 32 channels, 5 height, 7 width
        self.fc2 = nn.Linear(120, 10)  # Example output for 10 classes

    def forward(self, x):
        # Convolutional layers
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        
        # Flatten the output for the fully connected layer
        x = x.view(-1, 32 * 5 * 7)
        
        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_dataset, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {running_loss / 200:.4f}')
            running_loss = 0.0

print('Finished Training')

torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 20, 30])
torch.Size([32, 1, 2

In [11]:
def evaluate_model(model, test_dataloader):
    # Switch model to evaluation mode
    model.eval()

    # Initialize metrics
    correct = 0
    total = 0

    # Disable gradient computation
    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data

            images = images.to(device)
            labels = labels.to(device)

            # Forward pass to get outputs
            outputs = model(images)

            # Get predictions from the maximum value
            _, predicted = torch.max(outputs.data, 1)

            # Total number of labels
            total += labels.size(0)

            # Total correct predictions
            correct += (predicted == labels).sum().item()

    # Calculate accuracy
    accuracy = 100 * correct / total
    return accuracy

In [12]:
def evaluate_model_per_label(model, test_dataloader, device):
    # Switch model to evaluation mode
    model.eval()

    # Initialize dictionaries to store correct predictions and total counts for each label
    correct_pred = {label: 0 for label in range(4)}
    total_pred = {label: 0 for label in range(4)}

    # Disable gradient computation
    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data
            # Send data to the correct device
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass to get outputs
            outputs = model(images)

            # Get predictions from the maximum value
            _, predictions = torch.max(outputs, 1)

            # Collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[label.item()] += 1
                total_pred[label.item()] += 1

    # Calculate accuracy for each label
    label_accuracy = {label: 100 * correct_pred[label] / total_pred[label] if total_pred[label] > 0 else 0 for label in range(4)}
    return label_accuracy

In [13]:
test_dataloader = create_dataloader(testX, testY)
accuracy = evaluate_model(model, test_dataloader)
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

Accuracy of the model on the test images: 60.70%


In [14]:
label_accuracy = evaluate_model_per_label(model, test_dataloader, device)
for label in label_accuracy:
    print(f'Accuracy for label {label}: {label_accuracy[label]:.2f}%')

Accuracy for label 0: 0.00%
Accuracy for label 1: 0.00%
Accuracy for label 2: 0.00%
Accuracy for label 3: 100.00%


In [18]:
label_mapping = {
    'conflict': 0,
    'negative': 1,
    'neutral': 2,
    'positive': 3
}

list(label_mapping.keys())

['conflict', 'negative', 'neutral', 'positive']

In [31]:

import xml.etree.ElementTree as ET
from nltk.stem import SnowballStemmer
from nltk.parse.stanford import StanfordDependencyParser
import nltk.data  


In [32]:
inputfile = r'Dataset\semeval14\Restaurants_Train_v2.xml'

corpus=loadXML(inputfile)
texts=corpus.texts
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sent_num=[]
all_sents=[]
print('Clause begin')
for text in texts:
    sents=tokenizer.tokenize(text)  
    sents=[nltk.word_tokenize(sent) for sent in sents]
    all_sents.extend(sents)
    sent_num.append(len(sents))
print('Parse begin')
eng_parser = StanfordDependencyParser(r"B\stanford-parser-full-2020-11-17\stanford-parser.jar",
                                        r"B\stanford-parser-full-2020-11-17/stanford-parser-4.2.0-models.jar")

XML with 3041 sentences, 3693 AspectTerms, 1288 different AspectTerms
Clause begin
Parse begin


C:\Users\sam\AppData\Local\Temp\ipykernel_7668\3701193052.py:15: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  eng_parser = StanfordDependencyParser(r"B\stanford-parser-full-2020-11-17\stanford-parser.jar",


In [33]:
res_list=list(eng_parser.parse_sents(all_sents))
res_list=[list(i) for i in res_list]

In [54]:
depends=[]
for item in res_list:
    depend=[]
    for row in item[0].triples():
        print(row)
    break
    try:
        for row in item[0].triples():
            depend.append(row)
        depends.append(depend)
    except:
        print(item[0])


(('horrible', 'JJ'), 'cc', ('But', 'CC'))
(('horrible', 'JJ'), 'nsubj', ('staff', 'NN'))
(('staff', 'NN'), 'det', ('the', 'DT'))
(('horrible', 'JJ'), 'cop', ('was', 'VBD'))
(('horrible', 'JJ'), 'advmod', ('so', 'RB'))
(('horrible', 'JJ'), 'obl', ('us', 'PRP'))
(('us', 'PRP'), 'case', ('to', 'IN'))


In [44]:
for row in item[0].triples():
    print(row)

TypeError: 'NoneType' object is not subscriptable